## CNN

### Importar librerias generales

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
%matplotlib inline 

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import confusion_matrix, classification_report  

#### Establecemos los paths de los data set a utilizar

In [6]:
### SE HARA USO DE DATASET BALANCEADO PARA EL TRABAJO DE LOS MODELOS
path_dataset = "data_cleaned_balanced.csv"
#### IGUALMENTE SE DEJA EL DATASET NO BALANCEADO POR SI DESEA HACER MÁS PRUEBAS, solo se debe cambiar el path
path_dataset_NOBALANCEADO = "Data_Base_NOBALANCEADO.csv"

#### Leemos el archivo con los datos balanceados

In [7]:
data = pd.read_csv(path_dataset)

In [8]:
data.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Min,Bwd Pkt Len Min,Fwd Pkt Len Max,Bwd Pkt Len Max,Fwd Pkt Len Mean,Bwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Std,Flow Pkts/s,Flow Byts/s,Label,binaryLabel,benignoLabel,ransomwareLabel
0,4220250,11,5,209,17581,605,-1.0,52,-1,52.0,0.0,1.0,0.0,7.938877,1343.257988,Ransomware,0,0,1
1,0,7,5,2415,16696,321,-1.0,52,-1,52.0,0.0,0.0,0.0,0.000000,0.000000,Ransomware,0,0,1
2,2,13,4,2264,8590,585,-1.0,40,-1,40.0,0.0,1.0,0.0,0.259702,109.334732,Ransomware,0,0,1
3,0,22,1,255,21137,293,-1.0,83,-1,83.0,0.0,0.0,0.0,50.404073,18359.851480,Ransomware,0,0,1
4,0,1,0,420,0,420,-1.0,420,-1,420.0,0.0,0.0,0.0,0.000000,0.000000,Benigno,1,1,0


In [9]:
data.columns

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Min', 'Bwd Pkt Len Min',
       'Fwd Pkt Len Max', 'Bwd Pkt Len Max', 'Fwd Pkt Len Mean',
       'Bwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Std', 'Flow Pkts/s',
       'Flow Byts/s', 'Label', 'binaryLabel', 'benignoLabel',
       'ransomwareLabel'],
      dtype='object')

##### Establecemos cuales seran las columnas que seran utilizadas como labels y cuales seran la data para los modelos

In [10]:
columns_label = ['binaryLabel','benignoLabel','ransomwareLabel']
columns_data = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Min', 'Bwd Pkt Len Min',
       'Fwd Pkt Len Max', 'Bwd Pkt Len Max', 'Fwd Pkt Len Mean',
       'Bwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Std', 'Flow Pkts/s',
       'Flow Byts/s']

##### Separamos los valores que utilizaremos para los datos de entradas (Todas las columnas de datos) y los datos de salida (los labels)

In [11]:
dataX = data[columns_data].copy()
dataY = data[columns_label].copy()

In [12]:
dataX.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Min,Bwd Pkt Len Min,Fwd Pkt Len Max,Bwd Pkt Len Max,Fwd Pkt Len Mean,Bwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Std,Flow Pkts/s,Flow Byts/s
0,4220250,11,5,209,17581,605,-1.0,52,-1,52.0,0.0,1.0,0.0,7.938877,1343.257988
1,0,7,5,2415,16696,321,-1.0,52,-1,52.0,0.0,0.0,0.0,0.000000,0.000000
2,2,13,4,2264,8590,585,-1.0,40,-1,40.0,0.0,1.0,0.0,0.259702,109.334732
3,0,22,1,255,21137,293,-1.0,83,-1,83.0,0.0,0.0,0.0,50.404073,18359.851480
4,0,1,0,420,0,420,-1.0,420,-1,420.0,0.0,0.0,0.0,0.000000,0.000000


In [13]:
dataY.head()

,binaryLabel,benignoLabel,ransomwareLabel
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,1,1,0


#### GENERAMOS LOS DATOS DE ENTRENAMIENTO,TESTING Y VALIDACIÓN


#### Establecemos que las proporciones serán:
###### - ENTRENAMIENTO =  70%
###### - TESTING = 20%
###### - VALIDACION  = 10% 

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_ratio = 0.7
test_ratio = 0.2
validation_ratio = 0.1
semilla = 101

In [16]:
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=test_ratio, random_state=semilla)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_ratio/(validation_ratio + train_ratio), random_state=semilla)

##### Verificamos las separación de los datos

In [19]:
print('Train ' + str(X_train.shape[0]/dataX.shape[0]) + ' %')
print('Test ' + str(X_test.shape[0]/dataX.shape[0]) + ' %')
print('Validation ' + str(X_val.shape[0]/dataX.shape[0])+ ' %')

Train 0.6999919880354663 %
Test 0.20000534130968914 %
Validation 0.10000267065484457 %


#### Las dimensiones de cada una de las separaciones de datos

In [20]:
print('Data TRAIN: ', X_train.shape,'-', 'Label TRAIN: ' , y_train.shape)
print('Data TEST: ', X_test.shape,'-', 'Label TEST: ' , y_test.shape)
print('Data VALIDATION: ', X_val.shape,'-', 'Label VALIDATION: ' , y_val.shape)

Data TRAIN:  (104842, 15) - Label TRAIN:  (104842, 3)
Data TEST:  (29956, 15) - Label TEST:  (29956, 3)
Data VALIDATION:  (14978, 15) - Label VALIDATION:  (14978, 3)


#### Debido a necesidades del modelo CNN, lo valores de entrada (La Data) para entrenamiento, testing y validación tienen que ser del tipo 3 dimensional, por tal razón transformarmos esa matriz a una matriz trimensional generica

In [22]:
X_train_3D = X_train.values.reshape(X_train.shape[0],X_train.shape[1],1)
X_test_3D = X_test.values.reshape(X_test.shape[0],X_test.shape[1],1)
X_val_3D = X_val.values.reshape(X_val.shape[0],X_val.shape[1],1)

### Una vez listo lo anterior, comenzamos con el modelo CNN

#### Cabe señalar que se realizar 2 modelos, uno utilizará la función de perdida "SparseCategoricalCrossentropy" y la otra "BinaryCrossentropy"

***
1. SparseCategoricalCrossentropy → Para modelos de 2 o más clases de labels, deben ser enteros.(Por eso tienen que ser 0 y 1)
    1. Debe recibir como label un vector columna → Example : [0]
2. BinaryCrossentropy → Para modelos de 2 clases de label. (Por eso tienen que ser 0 y 1)
    1. Debe recibir como label un array2D → Example : [0 , 1]

### CNN - SparseCategoricalCrossentropy

In [23]:
model_scc = models.Sequential()

In [24]:
# Primera capa con dimensiones de entrada iguales al data
model_scc.add(layers.Conv1D(filters=15, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model_scc.add(layers.MaxPooling1D(pool_size=2))
model_scc.add(layers.Conv1D(filters=30, kernel_size=3, activation='relu'))
model_scc.add(layers.MaxPooling1D(pool_size=1))
model_scc.add(layers.Conv1D(filters=60, kernel_size=3, activation='relu'))
model_scc.add(tf.keras.layers.Dropout(0.2))
model_scc.add(layers.MaxPooling1D(pool_size=1))
model_scc.add(layers.Flatten())

In [25]:
model_scc.add(layers.Dense(32, activation='relu'))
model_scc.add(tf.keras.layers.Dropout(0.2))
model_scc.add(layers.Dense(10, activation='relu'))
model_scc.add(layers.Dense(2, activation='softmax'))

In [26]:
model_scc.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 13, 15)            60        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 6, 15)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 30)             1380      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 30)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 60)             5460      
_________________________________________________________________
dropout (Dropout)            (None, 2, 60)             0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2, 60)             0

In [27]:
model_scc.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy','accuracy'])

In [28]:
history_scc = model_scc.fit(X_train_3D, y_train['binaryLabel'], epochs=10, validation_data=(X_val_3D, y_val['binaryLabel']),shuffle=True)

Train on 104842 samples, validate on 14978 samples
Epoch 1/10
104842/104842 [==============================] - 15s 145us/sample - loss: 0.6334 - sparse_categorical_accuracy: 0.6794 - accuracy: 0.6794 - val_loss: 0.6242 - val_sparse_categorical_accuracy: 0.6891 - val_accuracy: 0.6891
Epoch 2/10
104842/104842 [==============================] - 15s 147us/sample - loss: 0.6305 - sparse_categorical_accuracy: 0.6828 - accuracy: 0.6828 - val_loss: 0.6152 - val_sparse_categorical_accuracy: 0.6981 - val_accuracy: 0.6981
Epoch 3/10
104842/104842 [==============================] - 20s 192us/sample - loss: 0.6270 - sparse_categorical_accuracy: 0.6862 - accuracy: 0.6862 - val_loss: 0.7681 - val_sparse_categorical_accuracy: 0.5451 - val_accuracy: 0.5451
Epoch 4/10
104842/104842 [==============================] - 19s 186us/sample - loss: 0.6767 - sparse_categorical_accuracy: 0.6366 - accuracy: 0.6366 - val_loss: 0.5997 - val_sparse_categorical_accuracy: 0.7135 - val_accuracy: 0.7135
Epoch 5/10
104842

#### Realizamos las predicciones con los datos de testeo

In [29]:
predictions_scc = model_scc.predict(X_test_3D)

In [30]:
predictions_scc

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [31]:
predictions_scc_argmax = np.argmax(predictions_scc, axis=1)

#### Aplicamos el reporte de clasificación y matriz de confusión

In [32]:
scnn_report_scc = classification_report(y_test['binaryLabel'], predictions_scc_argmax)  
print(scnn_report_scc)

              precision    recall  f1-score   support

           0       0.87      0.51      0.64     15005
           1       0.65      0.92      0.76     14951

    accuracy                           0.72     29956
   macro avg       0.76      0.72      0.70     29956
weighted avg       0.76      0.72      0.70     29956



In [33]:
scnn_cm_scc = confusion_matrix(y_test['binaryLabel'], predictions_scc_argmax)
print(scnn_cm_scc)

[[ 7610  7395]
 [ 1135 13816]]


### CNN - BinaryCrossentropy

In [34]:
model_bc = models.Sequential()

In [35]:
# Primera capa con dimensiones de entrada iguales al data
model_bc.add(layers.Conv1D(filters=15, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model_bc.add(layers.MaxPooling1D(pool_size=2))
model_bc.add(layers.Conv1D(filters=30, kernel_size=3, activation='relu'))
model_bc.add(layers.MaxPooling1D(pool_size=1))
model_bc.add(layers.Conv1D(filters=60, kernel_size=3, activation='relu'))
model_bc.add(tf.keras.layers.Dropout(0.2))
model_bc.add(layers.MaxPooling1D(pool_size=1))
model_bc.add(layers.Flatten())

In [36]:
model_bc.add(layers.Dense(32, activation='relu'))
model_bc.add(tf.keras.layers.Dropout(0.2))
model_bc.add(layers.Dense(10, activation='relu'))
model_bc.add(layers.Dense(2, activation='softmax'))

In [37]:
model_bc.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 13, 15)            60        
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 15)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 4, 30)             1380      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 4, 30)             0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 2, 60)             5460      
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 60)             0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 60)            

In [38]:
model_bc.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [39]:
history_bc = model_bc.fit(X_train_3D, y_train[['benignoLabel','ransomwareLabel']], epochs=10, validation_data=(X_val_3D, y_val[['benignoLabel','ransomwareLabel']]),shuffle=True)

Train on 104842 samples, validate on 14978 samples
Epoch 1/10
104842/104842 [==============================] - 18s 172us/sample - loss: 0.6667 - accuracy: 0.6728 - val_loss: 0.6605 - val_accuracy: 0.6853
Epoch 2/10
104842/104842 [==============================] - 18s 168us/sample - loss: 0.6520 - accuracy: 0.7025 - val_loss: 0.6543 - val_accuracy: 0.6978
Epoch 3/10
104842/104842 [==============================] - 18s 170us/sample - loss: 0.6568 - accuracy: 0.6929 - val_loss: 0.6515 - val_accuracy: 0.7034
Epoch 4/10
104842/104842 [==============================] - 18s 173us/sample - loss: 0.6592 - accuracy: 0.6880 - val_loss: 0.6714 - val_accuracy: 0.6636
Epoch 5/10
104842/104842 [==============================] - 18s 172us/sample - loss: 0.6621 - accuracy: 0.6822 - val_loss: 0.6566 - val_accuracy: 0.6931
Epoch 6/10
104842/104842 [==============================] - 18s 173us/sample - loss: 0.6546 - accuracy: 0.6972 - val_loss: 0.6410 - val_accuracy: 0.7245
Epoch 7/10
104842/104842 [=====

In [40]:
test_loss, test_acc = model_bc.evaluate(X_test_3D, y_test[['benignoLabel','ransomwareLabel']])

29956/29956 [==============================] - 2s 68us/sample - loss: 0.6560 - accuracy: 0.6943


In [41]:
print("PRECISIÓN: ",test_acc)
print("LOSS: ", test_loss)

PRECISIÓN:  0.69431835
LOSS:  0.6560453823084251


#### Realizamos las predicciones con los datos de testeo

In [42]:
predictions_bc = model_bc.predict(X_test_3D)

In [43]:
predictions_bc

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [44]:
predictions_bc_argmax = np.argmax(predictions_bc, axis=1)

#### Aplicamos el reporte de clasificación y matriz de confusión

In [45]:
scnn_report_cb = classification_report(y_test['binaryLabel'], predictions_bc_argmax)  
print(scnn_report_cb)

              precision    recall  f1-score   support

           0       0.37      0.57      0.45     15005
           1       0.09      0.04      0.06     14951

    accuracy                           0.31     29956
   macro avg       0.23      0.31      0.25     29956
weighted avg       0.23      0.31      0.25     29956



In [48]:
scnn_cm_cb = confusion_matrix(y_test['binaryLabel'], predictions_bc_argmax)
print(scnn_cm_cb)

[[ 8529  6476]
 [14323   628]]


In [47]:
### mejorar modelos se deben modificar las capas de los mismo

***